# 5. Redes convolucionales - De 101 a PRO

¡Bienvenidos a la quinta sesión! Tras haber visto en la sesión anterior las redes convolucionales y las técnicas más comunes para reducir el overfitting, hoy vamos a ver:

**Redes convolucionales**
* Extensiones al GD: Momentum, Adagrad, RMSprop, etc...
* Visualización de activaciones y filtros
* Transfer Learning y Fine-Tuning
* Data augmentation
* Arquitecturas más comunes

## 5.1 Arquitecturas comunes

Existen grupos de investigación que dedican su vida a desarrollar arquitecturas que funcionen y entrenarlas en datasets enormes, así que parece lógico aprovecharnos de esto en vez de intentar crear cada vez que tengamos un problema una arquitectura propia, verdad?

Esto, no solo nos va a ahorrar tiempo y dolores de cabeza, si no que nos va a aportar precisión y estabilidad!

Las arquitecturas más comunes hoy en día son:

* VGG
* ResNet
* Inception
* Xception

Vamos a verlas brevemente.

### VGG16 y VGG19

Esta arquitectura, que fue una de las primeras en aparecer, fue introducida por Simonyan y Zisserman en 2014 con su paper titulado Very Deep Convolutional Networks for Large Scale Image Recognition, que tenéis disponible aquí: https://arxiv.org/abs/1409.1556.

<img src="https://image.ibb.co/gkEOkJ/imagenet_vgg16.png" alt="imagenet_vgg16" border="0">

Se trata de una arquitectura bastante simple, usando solo bloques compuestos por un número incremental de capas convolucionales con filtros de tamaño 3x3. Además, para reducir el tamaño de los mapas de activación que se van obteniendo, se intercalan bloques maxpooling entre los convolucionales, reduciendo a la mitad el tamaño de estos mapas de activación. Finalmente, se utiliza un bloque de clasificación compuesto por dos capas densas de 4096 neuronas cada una, y una última capa, que es la de salida, de 1000 neuronas.

El 16 y 19 se refiere al número de capas con pesos que tiene cada red (las convolucionales y las densas, las de pooling no se cuentan). Se corresponden con las columnas D y E de la tabla a continuación.

<img src="https://image.ibb.co/cAyikJ/imagenet_vggnet_table1.png" alt="imagenet_vggnet_table1" border="0">

El motivo por el que veis el resto de arquitecturas en la tabla es porque, por aquel entonces, a Simonyan y Zisserman les costó bastante entrenar su arquitectura de forma que convergiera. Como no lo conseguían, lo que se les ocurrió fue entrenar primero redes con arquitecturas más sencillas, y una vez estas convergían y estaban entrenadas, aprovechavan sus pesos para inicializar la siguiente red, que era un poco más compleja, y así hasta llegar a la VGG19. Fijáos la importancia que tiene la inicialización de los pesos, como ya hemos visto. A este proceso se le conoce como "pre-training".

Sin emargo, esto fue en aquellos tiempos, ahora ya no se hace esto, ya que requiere de demasiado tiempo. Ahora podemos conseguir lo mismo utilizando la inicialización de Xavier/Glorot o de He et al., que ya vimos en la sesión 3.

Esta red, sin embargo, tiene un par de desventajas:

* Tarda muchísimo en entrenar
* Tiene un número muy elevado de parámetros

### ResNet

La arquitectura de ResNet, desarrollada por He et al. en 2015 (podéis ver su paper llamado "Deep Residual Learning for Image Recognition" aquí: https://arxiv.org/abs/1512.03385), supuso un hito al introducir un tipo de arquitectura exótica basada en "módulos", o como se conoce ahora, "redes dentro de redes".

Estas redes introdujeron el concepto de "conexiones residuales", que podéis ver en la siguiente imagen:

<center><img src="https://image.ibb.co/dXfUQJ/imagenet_resnet_residual.png" alt="imagenet_resnet_residual" border="0"></center>

Estos bloques lo que permiten es que llegue a la capa $l+1$ parte del mapa de activaciones previo sin modificar, y parte modificado por el bloque perteneciente a la capa $l$, como podéis ver en la imágen superior.

En 2016 mejoraron esta arquitectura incluyendo más capas en estos bloques residuales, como podéis observar en la siguiente imagen:

<center><img src="https://image.ibb.co/fmWEsy/imagenet_resnet_identity.png" alt="imagenet_resnet_identity" border="0"></center>

Existen variaciones de ResNet con distinto número de capas, pero la más usada es la ResNet50, que consta de 50 capas con pesos.

Es llamativo que aunque tiene muchas más capas que la VGG, necesita mucha menos memoria, casi 5 veces menos. Eso se debe a que esta red, en vez de capas densas en la etapa de clasificación, utiliza un tipo de capa que se llama GlobalAveragePooling, que lo que hace es convertir los mapas de activacones 2D de la última capa de la etapa de extracción de características a un vector de $n_{classes}$ que se utiliza para calcular la probabilidad de pertenecer a cada clase.

### Inception V3

Este tipo de arquitectura, que se introdujo en 2014 por Szegedy et al. en su paper llamado "Going Deeper with Convolutions" (https://arxiv.org/abs/1409.4842), utiliza bloques con filtros de diferentes tamaños que luego concatena para poder extraer características a diferentes escalas. Fijaos en la imagen:

<img src="https://image.ibb.co/mUJusy/imagenet_inception_module.png" alt="imagenet_inception_module" border="0">

Para que lo entendáis, la meta del bloque "inception" es calcular mapas de activaciones con convoluciones de 1x1, 3x3 y 5x5 para conseguir extraer características a diferentes escalas. Luego simplemente se concatenan todos estos mapas de activaciones en uno solo.

Esta arquitectura necesita incluso menos memoria que la VGG y la ResNet.

### Xception

Esta arquitectura la propuso François Chollet (el creador de Keras) y lo unico que aporta respecto a Inception es que realiza las convoluciones de una forma óptima para que tarden menos tiempo. Esto lo consigue separando las convoluciones 2D en 2 convoluciones 1D. Si tenéis interés en saber más, aquí tenéis el paper: "Xception: Deep Learning with Depthwise Separable Convolutions", https://arxiv.org/abs/1610.02357.

En términos de memoria es muy similar a la Xception, y este es el esquema de su arquitectura:

<img src="https://image.ibb.co/dpKpQJ/imagenet_xception_flow.png" alt="imagenet_xception_flow" border="0">

Por último, merece la pena hablar también de la SqueezeNet.

### SqueezeNet

Esta red es extremadamente ligera (sus pesos ocupan 5MB, en comparación de los 500MB de la VGG, o los 100 de la Inception, por ejemplo) y consigue un accuracy de ~57% rank-1 o ~80% rank-5 con el ImageNet.

¿Qué significa rank-1 y rank-5, o top-1 y top-5? 

* rank-1 accuracy: comparamos si la clase con la mayor probabilidad según nuestra red acierta a la etiqueta real
* rank-5 accuracy: comparamos si una de las 5 clases con mayor probailidad según nuestra red acierta a la etiqueta real

¿Cómo consigue esta red ocupar tan poco y a la vez ser precisa? Pues lo consigue utilizando una arquitectura que "comprime" los datos y después los expande, tal y como podéis ver en la siguiente imagen:

<img src="https://image.ibb.co/dQjpQJ/imagenet_squeezenet_fire.png" alt="imagenet_squeezenet_fire" border="0">


### Comparación de tamaños

<img src="https://image.ibb.co/kUF5jd/net_param_size.png" alt="net_param_size" border="0">

### Comparación del accuracy que consiguen vs. número de parámetros

<img src="https://image.ibb.co/fAqo4d/net_comp_acc_vs_params.png" alt="net_comp_acc_vs_params" border="0">

### Fuentes:

* https://www.pyimagesearch.com/2017/03/20/imagenet-vggnet-resnet-inception-xception-keras/
* https://towardsdatascience.com/neural-network-architectures-156e5bad51ba
* https://medium.com/@siddharthdas_32104/cnns-architectures-lenet-alexnet-vgg-googlenet-resnet-and-more-666091488df5

Tenéis que saber que hay infinitas arquitecturas, pero estas son con diferencia las más usadas. Normalmente, ante un problema, no vamos a ponernos a definir nuestra arquitectura, sino que usaremos una de estas arquitecturas.

Vale, pues ahora que ya las habéis visto, vamos a ver cómo podemos implementarlas en Keras

### Ejemplo de clasificación de imágenes con VGG, ResNet, Inception y Xception con Keras

In [0]:
# importamos los paquetes necesarios


## 5.2 Extensiones al GD

Hasta ahora hemos visto 3 métodos para implementar el backpropagation:

* Gradient Descent
* Stochastic Gradient Descent
* Mini-Batch Stochastic Gradient Descent

De los cuales, nos quedamos con el mini-batch porque permite una mayor rapidez, al no tener que calcular los gradientes y errores para todo el dataset, y elimina la alta variabilidad existente en el SGD.

Bueno, pues existen mejoras sobre estos métodos, como el **momentum**. Además, hay otros algoritmos más complejos como el Adam, RMSProp o el Adagrad.

Vamos a verlos!

<!-- TODO: http://cs231n.github.io/neural-networks-3/#update-->

### 5.2.1 Momentum

Imáginad que volvéis a ser un crio y que se os ha ocurrido la genial idea de poneros vuestros patines, subiros a lo alto de la calle más empinada y empezar a bajarla. Ni que decir tiene que sois totalmente principiantes y esta es la segunda vez que os calzáis unos patines.

No sé si alguno de vosotros habrá hecho esto realmente alguna vez, pero bueno, yo sí, así que os voy a explicar lo que pasa:

* nada más empezar, la velocidad es pequeña, incluso parece que controlas y que podrías parar en cualquier momento
* pero conforme más bajas, más velocidad coges: **a esto se le llama momento (momentum)**
* con lo cual, conforme más carretera bajas, más *inercia* llevas y más rápido recorres los metros

Bueno, para aquellos que seáis curiosos, el final de la historia es que al final de la calle empinada hay una valla. El resto os lo podéis imaginar...

Pues bien, el momento es precisamente esto. Conforme vamos bajando en nuestra curva de pérdidas al calcular los gradientes y hacer las actualizaciones, más importancia le damos a las actualizaciones que van en la dirección que minimizan el gradiente, y menos importancia a las que van en otras direcciones. 

<img src="https://image.ibb.co/e5DyHd/opt_momentum.png" alt="opt_momentum" border="0">

Así, lo que conseguimos es acelerar el entrenamiento de la red. 

Además, gracias al momento, podría haber sido capaz de evitar pequeños baches o agujeros en la carretera (volar sobre ellos gracias a mi velocidad). 

TEORÍA: http://cs231n.github.io/neural-networks-3/#sgd

### 5.2.2 Nesterov momentum

Volviendo al ejemplo de antes: estamos bajando la carretera a toda velocidad (porque hemos construído mucho momento) y de repente vemos al final la vaya. Nos gustaría ser capaces de frenar, de reducir la velocidad para no estamparnos. Pues esto es precisamente lo que hace Nesterov.

Lo que hace Nesterov es calcular el gradiente, en vez de en el punto actual, en el punto en el que sabemos que nuestro momento va a llevarnos, para luego aplicar una corrección.

<!-- <img src="https://image.ibb.co/fWsscd/opt_momentum_nesterov.png" alt="opt_momentum_nesterov" border="0">-->

<img src="https://image.ibb.co/eAPBnd/opt_momentum_nesterov_2.png" alt="opt_momentum_nesterov_2" border="0">

Fijaos: usando el momento estándar, calculamos el gradiente (vector azul pequeño) y luego damos un gran paso en la dirección del gradiente (vector azul grande).

Usando Nesterov, primero daríamos un gran salto en la dirección de nuestro gradiente previo (vector marron), mediriamos el gradiente y haríamos la corrección oportuna (vector rojo).

En la práctica, funciona un poco mejor que el momento a solas. Es como calcular el gradiente de los pesos en el futuro (porque les hemos sumado el momento que habíamos calculado).

TEORÍA: http://cs231n.github.io/neural-networks-3/#sgd




Tanto el momento como el momento de Nesterov son extensiones del SGD.

Lo que vamos a ver ahora están basadas en learning rates adaptativos, lo que nos permite acelerar o frenar la *velocidad* con la que actualizamos los pesos. Por ejemplo, podríamos usar una velocidad alta al principio, e ir bajándola conforme nos aproximásemos al mínimo.

### 5.2.3 Adaptive gradient (AdaGrad)

Lleva un historial de los gradientes calculados (en concreto, de la suma de los gradientes al cuadrado) y normaliza el "paso" de la actualización.

La idea es que los parámetros que tengan un gradiente muy alto, y por tanto, la actualización de sus pesos fuese a ser brusca, se les asignará un learning rate bajo.

A la misma vez, los parámetros que tengan un gradiente muy bajo, se les asignará un learning rate alto.

Así conseguimos acelerar la convergencia.

PAPER: http://jmlr.org/papers/v12/duchi11a.html

### 5.2.4 RMSprop

El problema de AdaGrad es que al calcular la suma de los gradientes al cuadrado, estamos usando una funcion monotónica creciente, lo que puede ocasionar que el learning rate trate de compensar valores que no dejan de crecer hasta que se hace cero, con lo que deja de aprender.

Lo que propone RMSprop, que no está publicado, pero podéis leer más sobre él aquí: http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf, es decrementar esa suma de los gradientes al cuadrado usando un `decay_rate`.


### 5.2.5 Adam

Por último, Adam es uno de los algoritmos más modernos, que mejora el RMSprop añadiendo momento a la regla de actualización. Introduce 2 nuevos parámetros, `beta1` y `beta2`, con valores recomendados de 0.9 y 0.999.

PAPER: https://arxiv.org/abs/1412.6980

### Pero mejor vamos a lo que nos interesa, cuál tengo que utilizar?

<img src="https://image.ibb.co/mjHstJ/opts_comp.png" alt="opts_comp" border="0">

**Como regla general, empezad con Adam.** Si no funciona bien, ya os calentaréis la cabeza.

+INFO: 
* https://deepnotes.io/sgd-momentum-adaptive
* http://ruder.io/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms

## 5.3 Visualización de activaciones y filtros

La mayoría de la gente considera que las redes neuronales son una caja negra, pero eso es por dos cosas:

* no entienden el back-propagation
* nunca han visualizado *el estado* de sus redes neuronales

Nosotros ya sabemos como funciona el backprop, pero todavía no hemos visualizado nada. Así que vamos a por ello!

Primero, qué es lo que podemos visualizar?

* las activaciones de las activaciones de las capas
* los filtros de nuestras capas convolucionales
* qué imágenes dan la máxima activación para cada clase de nuestro dataset

### 5.3.1 Visualización de las activaciones de las capas

Esto es de lo mejorcito que podemos hacer para entender cómo funciona nuestra red.

Las activaciones de las capas son simplemente los resultados que obtenemos a la salida de cada capa durante la forward pass. Normalmente, cuando visualizamos las activaciones de una red con activaciones de tipo ReLU, necesitamos unas cuantas épocas antes de empezar a ver algo útil.

Una cosa para la que son muy útiles es para ver si algún filtro está completamente negro para diferentes entradas, es decir, todos sus elementos son siempre 0. Esto significa que el filtro está *muerto*, y normalmente pasa cuando entrenamos con learning rates altos.

Aquí podéis ver unos ejemplos, aunque luego lo vamos a hacer nosotros:

<img src="https://image.ibb.co/j1Jkfy/vis_act1.jpg" alt="vis_act1" border="0" height="400">
<img src="https://image.ibb.co/eauQfy/vis_act2.jpg" alt="vis_act2" border="0" height="400">

Estos ejemplos se corresponden con las activaciones típicas de la primera capa convolucional (izqda) y de la quinta (dcha) de la red AlexNet entrenada cuando ve una imagen de un gato.


### 5.3.2 Visualización de los filtros de las capas convolucionales

Otra cosa que podemos visualizar para ver cómo funciona nuestra red neuronal convolucional, son los filtros que ha aprendido. Normalmente, estos filtros son más interpetables en las primeras capas de la red que en las últimas. Sobretodo, es útil visualizar los filtros de la primera, que está mirando directamente a las imágenes de entrada.

Fijaos, a continuación podéis ver los filtros de la primera y la segunda capa convolucional de la AlexNet.

<img src="https://image.ibb.co/g2H0DJ/vis_filt1.jpg" alt="vis_filt1" border="0" height="400">
<img src="https://image.ibb.co/h9L97d/vis_filt2.jpg" alt="vis_filt2" border="0" height="400">

Y quizás os estéis preguntando que de qué os sirve visualizar estos filtros, verdad?

Pues porque normalmente, una red bien entrenada tendrá filtros perfectamente definidos, al menos en las primeras capas, y sin practicamente ruido. Si os fijáis, es el caso de la imagen de ejemplo.

Si por el contrario tuviésemos filtros con mucho ruido podría deberse a que hace flata entrenar más la red, o a que tenemos overfitting y necesitamos algún método de regularización.

Los filtros de la segunda capa son menos indicativos, pero aún así, se puede intuir que no tienen ruido. Esto lo veremos mejor con el ejemplo que haremos luego.

### 5.3.3 Visualización de las imágenes que más activan una determinada neurona (=clase de nuestro dataset)

Por último, otra cosa que puede ayudaros a decidir si vuestra red está funcionando bien o no es visualizar las imágenes que más activan cada neurona de salida, lo que equivale a visualizar las imágenes que más se acercan a cada clase de vuestro dataset.

Para ello, se suele coger un buen dataset de imágenes y pasárlas a la red mientras se lleva un histórico de qué imágenes activan más una determinada neurona. Así, luego podemos visualizar las imágenes que más han activado esa neurona, además de su *receptive field*.

A continuación podéis ver las imágenes que más activan algunas de las neuronas de la capa POOL5 de AlexNet. El recuadro blanco muestra el receptive field.

<img src="https://image.ibb.co/cdPfDJ/vis_pool5max.jpg" alt="vis_pool5max" border="0" height="400">

Podéis apreciar como el campo receptivo es bastante grande, y que algunas neuronas reaccionan más a partes de cuerpo, textos, etc.


## Veamos todo esto con un ejemplo!

## Ejemplo: visualizando activaciones de capas y filtros


### Vamos primero a visualizar las activaciones de la última capa (saliency)

Para ello, necesitamos cambiar la activación de la última capa, de softmax a lineal, para una correcta visualización.

In [0]:
# visualizar filtros

A la función `visualize_saliency` tenemos que pasarle el modelo, el id de la capa, el id de la clase para la que queremos ver las activaciones, y la imagen para la que queremos ver las activaciones.
    
Y qué es eso del id de la clase para la que queremos ver las activaciones? pues que en el caso de la VGG16 coon los pesos de la ImageNet, la clase pájaro es la 20, por lo cual, si le metemos una imagen de un pájaro, debería activarse bastante, e indicarnos en qué se fija para decidir que efectivamente es un pájaro. Si le metiésemos un 64, buscaría una green mamba, que es una serpiente por lo que las activaciones deberían ser mucho menores.

<img src="http://www.craterlakeinstitute.com/natural-history/images/dipper.jpg" border="0" height="180">
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/74/Eastern_Green_Mamba_02.jpg/220px-Eastern_Green_Mamba_02.jpg" border="0" height="180">

Tenéis el listado completo de las 1000 clases aquí: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

Vamos entonces a ver las activaciones para nuestras dos imágenes de prueba!

Fijáos como cuando intentamos ver las activaciones para otra clase que no está presente en la imagen, lo que obtenemos es... nada.

Vamos a probar otro método de visualización: el cam-saliency.

En este caso, la visualización contiene más detalles, ya que hace uso de la información no solo de la capa indicada, si no de la anterior capa Conv o Pool que encuentre.

PAPER: https://arxiv.org/pdf/1610.02391v1.pdf

Fijáos qué preciosidad:

### Veamos los filtros!

Genial!! Pues vamos a ver ahora los filtros, no?!

Vamos a ver las visualizaciones de los filtros de la primera capa convolucional!

Vamos a ver ahora filtros de diferentes capas:

## 5.4 Transfer Learning y Fine-Tuning

Creo que todos coincidiréis conmigo en que entrenar una red neuronal lleva su tiempo, verdad? Y si os dijera que hay formas de evitar tener que:

* definir la arquitectura de una red neuronal
* entrenarla desde el principio

Las formas de evitar tener que definir la arquitectura las veremos al final de la sesión, y consiste en utilizar arquitecturas predefinidas y que se sabe que funcionan bien: ResNet, AlexNet, VGG, Inception, DenseNet, etc.

Y lo de evitar entrenarla de cero? A qué me refiero con eso?

Pues bien, como ya sabéis, las redes se inicializan con unos pesos aleatorios (normalmente) que tras una serie de épocas consiguen tener unos valores que permiten clasificar adecuadamente nuestras imágenes de entrada. 

¿Qué pasaría si pudiésemos inicializar esos pesos a unos valores que nosotros supieramos que ya son buenos para clasificar un determinado dataset?

De esta forma, no necesitaríamos ni un dataset tan grande como el necesario si queremos entrenar una red de cero (de cientos de miles o incluso millones de imágenes podríamos pasar a unas pocas miles) ni necesitaríamos esperar un buen número de épocas a que los pesos cogiesen valores buenos para la clasificación, lo tendrían mucho más fácil debido a su inicialización.

Vamos a ver cuales son las diferencias entre el transfer learning y el fine-tuning:

### 5.4.1 Transfer Learning 

Pongamos como ejemplo la red VGG16 entrenada sobre el dataset ImageNet. Veamos su arquitectura:

<img src="https://image.ibb.co/h9Gpay/imagenet_vgg16.png" alt="imagenet_vgg16" border="0">

Sabemos que ImageNet consiste en un dataset de más o menos 1,2 millones de imágenes para entrenamiento, 50.000 para validación y 100.000 para test, pertenecientes a 1000 categorías.

Ahora imaginad que lo que queremos nosotros es aplicar la VGG16 entrenada sobre ImageNet a nuestro problema, que es el CIFAR-10. Como podríamos hacerlo?

Bien, si recordáis el esquema general de una CNN, lo que teníamos era un extractor de características en la primera etapa  y después un clasificador, verdad? Mirad:

<img src="https://image.ibb.co/kXj0cd/cnn_feat_class.jpg" alt="cnn_feat_class" border="0" height="250">

Pues qué os parecería si quitásemos la última capa de la VGG16, que simplemente lo que hace es sacar una probailidad para cada una de las 1000 clases del ImageNet y la sustituyésemos por una capa que sacase 10 probabilidades? De esta forma, podríamos aprovechar todo el conocimiento que tiene la VGG16 entrenada sobre el ImageNet y aplicarlo a nuestro problema!

Veamos cómo podríamos hacerlo:

¿Sorprendente, no creéis? No hemos tenido que entrenar nada, y hemos unos resultados nada malos!

De hecho, cuanto más parecidos sean el dataset sobre el que ha sido entrenada originalmente la red y el dataset de nuestro problema, mejores resultados obtendremos.

**Y si nuestro dataset no tiene nada que ver con el de ImageNet o queremos mejorar aún más los resultados?**

Pues para eso, tenemos el **fine tuning**.


## 5.3.2 Fine tuning

Con el fine-tuning, primero cambiamos la última capa para que coincida con las clases de nuestro dataset, como hemos hecho antes con el transfer learning. Pero además, también **reentrenamos** las capas de la red que queramos.

Por ejemplo, recordad la arquitectura de la VGG16:

<img src="https://image.ibb.co/h9Gpay/imagenet_vgg16.png" alt="imagenet_vgg16" border="0">

Una posible opción sería cambiar la última capa a una de 10 neuronas (nuestro CIFAR 10 tiene 10 clases) y luego re-entrenar la red permitiendo que se modifiquen los pesos de las capas fully connected, es decir, de la etapa de clasificación. Para esto, inicializariamos nuestra red con los pesos del ImageNet, y luego congelaríamos todas las capas convolucionales y de max pooling para que no modificasen sus pesos, dejando solo libres las fully connected.

Una vez hecho eso, empezaríamos a re-entrenar. De esta forma, logramos aprovechar la etapa de extracción de características de nuestra red, y solamente *afinamos* el clasificador final para que funcione mejor con nuestro dataset.

De hecho, este enfoque se puede hacer también guardándonos las características que da la última capa de max pooling, y luego metiendo esos datos a cualquier clasificador (SVM, logreg, etc).

Pero no estamos limitados a re-entrenar solo las fully connected, si queremos, podemos re-entrenar también la etapa de extracción de características, es decir, las capas convolucionales y de pooling.

Es imporante que tengáis en cuenta lo que hablamos cuando vimos las capas convolucionales: en una red, las primeras capas detectan patrones más sencillos y generales, y cuanto más avanzamos en la arquitectura, más específicos al dataset y más complicados van siendo los patrones que detectan.

Por lo tanto, podríamos por ejemplo permitir también que se re-entrenase el último bloque de convolucionales y pooling.

### Cuándo hago fine tuning y cuando transfer learning? Cómo elijo desde qué capa re-entrenar?

Bien, como norma general, lo primero que haremos será transfer learning, es decir, no re-entrenaremos nuestra red. Eso nos dará un *baseline* que tendremos que superar. Después, re-entrenaremos solo la etapa de clasificación, y después podemos probar a re-entrenar también algún bloque convolucional.

**Receta**
* Hacer transfer learning, es decir, modificar solo la última capa para que tenga el número de salidas igual a nuestras clases (*baseline*)
* Probar a re-entrenar la etapa de clasificación, es decir, las capas densas
* Probar a re-entrenar alguna etapa convolucional

Siguiendo esos pasos la mayoría de las veces llegaréis a un resultado adecuado para vuestro problema ;-)

También depende del tipo de problema que tengáis. Por ejemplo, si...

* **el nuevo dataset es pequeño y parecido al original**: cuidado al hacer fine-tuning, quizás sea mejor escoger las características de la ultima capa de la etapa convolucional y usar un SVM o clasificador lineal
* **el nuevo dataset es grande y parecido al original**: al tener más datos probablemente no incurramos en over-fitting, así que podemos hacer fine-tuning con más confianza
* **el nuevo dataset es pequeño y muy diferente al original**: lo mejor sería usar características de una capa más temprana de la etapa convolucional, ya que ésta se fijará en patrones más generales que las últimas capas, y luego emplear un clasificador lineal
* **el nuevo dataset es grande y muy diferente al original**: dale matraca a esa red, entrenala desde el principio! o como se dice en inglés, from scracth! De todas formas, sigue siendo recomendable que inicialices los pesos con los del ImageNet.


**NOTA:**

Cuando uséis alguna de estas técnicas debéis tener en cuenta las posibles restricciones de los modelos preentrenados. Por ejemplo, pueden exigir un tamaño mínimo de imagen. Además, cuando se re-entrenan redes, se suelen escoger learning rates más bajos que si lo hacemos desde cero, ya que partimos de una inicialización de pesos que se presupone buena.


### Vamos a ver un ejemplo de fine tuning!

**Esto que acabo de contaros es el santo grial del deep learning, que no se os olvide! Repetid conmigo: **

### El fine tuning es mi mejor amigo. Nunca pasaré de él. Empezaré con un baseline que luego intentaré mejorar.

Perfecto! :-)

## 5.4 Data augmentation

Acabamos de ver el fine tuning, que es muy útil cuando nuestros datasets no son muy grandes y estamos limitados por el número de imágenes que tenemos disponibles. Pero no os he contado toda la verdad...

**hay otra forma, muy potente, de conseguir mejores resultados, sobretodo si nuestro dataset es limitado!**

Esta técnica se llama **data augmentation**, probablemente porque lo que hace es sacar más datos a partir de los que tenemos disponibles. Básicamente lo que hacemos es girar/trasladar/emborronar/cambiar el color/acercar o alejar/lo que se os ocurra las imagenes que tenemos. De esta forma, damos lugar a muchas más imágenes de las que teníamos inicialmente.

Ahora algún avispado o avispada podría preguntarme: pero y cómo sabemos las clases de esas nuevas imágenes generadas?

Pues es una muy buena pregunta! De hecho, es algo que tenemos que tener muy en cuenta: **las nuevas imágenes toman la clase de la imagen origen**, con lo cual, **las transformaciones no pueden ser muy exageradas, y mucho menos ocasionar que se parezcan a otra clase!!**

Para llevar a cabo este proceso existe una librería maravillosa en python que se llama imgaug (https://github.com/aleju/imgaug):

<img src="https://raw.githubusercontent.com/aleju/imgaug/master/examples_grid.jpg">